# Geohashing Census Data

In [2]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
from censusUtils import getCensusPoints, getCensusData, getCensusName
basedir="/Users/tgadf/Downloads/Census"

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-10-30 13:36:10.460552


# Central Census Processing

In [5]:
def processShapefile(stdir, debug=False, keepRecords=True, keepGeos=True):
    from os.path import join
    from timeUtils import clock, elapsed

    todo = {"Records": keepRecords, "Geos": keepGeos}

    for stdir in stdirs:

        geoShapeMap = {}
        shapeData = {}

        sf = shapefile.Reader(join(basedir, stdir, stdir))

        ipop,ihouse,ihouseocc = getCensusPoints(sf, show=False)

        startall,cmtall = clock("Geohashing all {0} shapes".format(stdir))
        ngeos = 0
        irec  = 0
        areas = []
        for shapeRec in sf.iterShapeRecords():
            irec += 1

            ####################################################################
            ## Record
            ####################################################################
            record = shapeRec.record
            geoid  = record[0]
            state  = geoid[:2]
            name   = record[1]
            memi   = record[2]
            area   = record[-1]
            pops   = record[ipop:ipop+19]
            house  = record[ihouse:ihouse+15]
            occpy  = record[ihouseocc:ihouseocc+9]

            name,memi = getCensusName(stdir, name, memi)
            recdata   = getCensusData(pops, house, occpy)

            if geoid in ['0201601615']:
                continue

            if todo["Records"] is True:
                shapeData[geoid]   = {"Name": name}
                shapeData[geoid].update(recdata)
                if irec % 100 == 0:
                    print(irec,'\t',geoid,'\t',name,'\t',area)



            if todo["Geos"] is True:
                ####################################################################
                ## Geohashs (Shape)
                ####################################################################
                shape = shapeRec.shape
                maxPrec = 7
                if area > 10:
                    maxPrec = 4
                elif area > 2:
                    maxPrec = 5
                elif area > 0.5:
                    maxPrec = 6
                areas.append(area)

                if debug:
                    start, cmt = clock("Geohashing Shape with name {0}".format(name))

                geos  = getGeos(shape, prec=6, returnKeys=True)
                #geos  = getShapeAndGeos(shape, maxPrec=maxPrec, debug=False)
                if debug:
                    elapsed(start, cmt)
                geoShapeMap[geoid] = geos
                print(irec,'\t',geoid,'\t',name,'\t',area,'\t--->',len(geos))
                if len(geos) <= 2:
                    print("---------------------> {0} has less than 2 geoids <---------------------".format(geoid))
                ngeos += len(geos)


            if irec % 250 == 0 and False:
                print("Found {0} records with {1} geos of type {2}".format(len(shapeData), ngeos, stdir))
                saveGeoData(shapeData, geoShapeMap, 0, ngeos, "Census/{0}-{1}-tmp".format(stdir, prec))


        print("Found {0} records with {1} geos of type {2}".format(len(shapeData), ngeos, stdir))
        saveGeoData(shapeData, geoShapeMap, 0, ngeos, "Census/{0}-{1}".format(stdir, prec))


        elapsed(startall,cmtall)


In [6]:
processShapefile("ZCTA_2010Census_DP1")

NameError: name 'stdirs' is not defined